In [1]:
!pip install numpy

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np

missingness = lambda dataframe: dataframe.isna().sum() / dataframe.shape[0] * 100

In [3]:
df = pd.read_csv('bfro_reports_geocoded.csv')

df.head()

,observed,location_details,county,state,season,title,latitude,longitude,date,number,...,moon_phase,precip_intensity,precip_probability,precip_type,pressure,summary,uv_index,visibility,wind_bearing,wind_speed
0,Ed L. was salmon fishing with a companion in P...,East side of Prince William Sound,Valdez-Chitina-Whittier County,Alaska,Fall,NaN,NaN,NaN,NaN,1261.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,heh i kinda feel a little dumb that im reporti...,"the road is off us rt 80, i dont know the exit...",Warren County,New Jersey,Fall,NaN,NaN,NaN,NaN,438.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,I was on my way to Claremont from Lebanon on R...,Close to Claremont down 120 not far from Kings...,Sullivan County,New Hampshire,Summer,Report 55269: Dawn sighting at Stevens Brook o...,43.41549,-72.33093,2016-06-07,55269.0,...,0.10,0.001,0.7,rain,998.87,Mostly cloudy throughout the day.,6.0,9.70,262.0,0.49
3,I was northeast of Macy Nebraska along the Mis...,Latitude & Longitude : 42.158230 -96.344197,Thurston County,Nebraska,Spring,Report 59757: Possible daylight sighting of a ...,42.15685,-96.34203,2018-05-25,59757.0,...,0.38,0.000,0.0,NaN,1008.07,Partly cloudy in the morning.,10.0,8.25,193.0,3.33
4,"While this incident occurred a long time ago, ...","Ward County, Just outside of a the Minuteman T...",Ward County,North Dakota,Spring,Report 751: Hunter describes described being s...,48.25422,-101.31660,2000-04-21,751.0,...,0.60,NaN,NaN,rain,1011.47,Partly cloudy until evening.,6.0,10.00,237.0,11.14


In [4]:
print('missingness')
missingness(df)

missingness


observed               0.758374
location_details      15.546661
county                 0.000000
state                  0.000000
season                 0.000000
title                 20.012640
latitude              20.012640
longitude             20.012640
date                  20.012640
number                 0.000000
classification         0.000000
geohash               20.012640
temperature_high      34.737729
temperature_mid       37.855488
temperature_low       37.792290
dew_point             34.000421
humidity              34.000421
cloud_cover           39.898884
moon_phase            33.515905
precip_intensity      47.482621
precip_probability    47.482621
precip_type           67.137139
pressure              49.210027
summary               33.515905
uv_index              33.515905
visibility            40.657257
wind_bearing          33.705498
wind_speed            33.663366
dtype: float64

In [5]:
print('numbers all unique')
len(df.number.unique()) == df.shape[0]

numbers all unique


True

# Clean bigfoot data

In [6]:
# clean county names to better match with county db
df['county'] = df.county.str.replace('County','')
df['county'] = df.county.str.strip()

# Impute

In [7]:
missing_loc = df[df.latitude.isna() | df.longitude.isna()]

# Save just ones without location

In [8]:
has_loc_df = df.drop(missing_loc.index, axis=0)

In [9]:
has_loc_df.to_csv('sightings_with_location.csv', index=False)
# has_loc_df.head()

# impute county centroid for location

In [10]:
# https://en.wikipedia.org/wiki/User:Michael_J/County_table
counties = pd.read_csv('counties.txt',delimiter='\t')
counties = counties.rename({'County [2]': 'County'},axis=1)
counties.isna().sum()

Sort [1]              0
State                 0
FIPS                  0
County                0
County Seat(s) [3]    0
Population (2010)     0
Land Area km²         0
Land  mi²             0
Water Area km²        0
Water Area mi²        0
Total Area km²        0
Total Area mi²        0
Latitude              0
Longitude             0
dtype: int64

In [11]:
initials = pd.read_csv('state_initials.txt', delimiter='\t', names=['state', 'abbrev', 'initial'])
initials.head()

,state,abbrev,initial
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [12]:
# get the state's full name to match on with bigfoot df
counties = counties.merge(initials, left_on='State', right_on='initial')

counties = counties.drop(['Sort [1]', 'State', 'abbrev'],axis=1)
missingness(counties)

FIPS                  0.0
County                0.0
County Seat(s) [3]    0.0
Population (2010)     0.0
Land Area km²         0.0
Land  mi²             0.0
Water Area km²        0.0
Water Area mi²        0.0
Total Area km²        0.0
Total Area mi²        0.0
Latitude              0.0
Longitude             0.0
state                 0.0
initial               0.0
dtype: float64

## Match the sightings with missing coordinates based on state & county
Using the county centroid coordinates

In [14]:
# Keep old indicies
missing_loc['sighting_idx'] = missing_loc.index

county_info = ['state', 'County', 'Latitude', 'Longitude']

imputed_locs = pd.merge(missing_loc, counties[county_info], 
         left_on=['state', 'county'], 
         right_on=['state', 'County'], 
         how='inner', suffixes=['bf','cnt'])

imputed_locs = imputed_locs.set_index('sighting_idx')
imputed_locs.tail()

<ipython-input-14-955cfd435fc1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_loc['sighting_idx'] = missing_loc.index


,observed,location_details,county,state,season,title,latitude,longitude,date,number,...,precip_type,pressure,summary,uv_index,visibility,wind_bearing,wind_speed,County,Latitude,Longitude
sighting_idx,,,,,,,,,,,,,,,,,,,,,
4741,It was last summer July 1998. I was sleeping w...,Jeffersonville In. 47130 U.S.A. Clark County H...,Clark,Indiana,Summer,NaN,NaN,NaN,NaN,2475.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Clark,+38.476217°,–85.711122°
4742,My cousin and I were camping way out in the wo...,"Indiana, Brown County, Elkinsville, Lake Monro...",Brown,Indiana,Spring,NaN,NaN,NaN,NaN,2460.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brown,+39.192585°,–86.239410°
4743,While backpacking near the horse trails and ac...,Near Bedford south of Brown County in the Hoos...,Brown,Indiana,Winter,NaN,NaN,NaN,NaN,2461.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brown,+39.192585°,–86.239410°
4745,My wife and I were driving to Indianapolis to ...,On Interstate 65 in Indiana somewhere around t...,Boone,Indiana,Winter,NaN,NaN,NaN,NaN,2459.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Boone,+40.050892°,–86.469014°
4746,It was about 7:00 PM in September. It was stil...,"Blackford County, Indiana located in the south...",Blackford,Indiana,Summer,NaN,NaN,NaN,NaN,2458.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blackford,+40.472672°,–85.323730°


In [15]:
imputed_locs['is_estimated'] = True

In [17]:
clean = lambda series: series.str.replace('\+','') \
    .str.replace('°','') \
    .str.replace('–','-') \
    .astype('float')

imputed_locs['latitude'] = clean(imputed_locs.Latitude)
imputed_locs['longitude'] = clean(imputed_locs.Longitude)

# imputed_loc = imputed_loc.drop(county_info, axis=1)

<ipython-input-17-5e469b8bba43>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  clean = lambda series: series.str.replace('\+','') \


In [18]:
imputed_locs.latitude

sighting_idx
1       40.853524
32      40.853524
9       45.037929
11      43.427239
22      43.427239
          ...    
4741    38.476217
4742    39.192585
4743    39.192585
4745    40.050892
4746    40.472672
Name: latitude, Length: 921, dtype: float64

## Replace NaN coordinates with county centroids

In [35]:
# fill in values
df.loc[imputed_locs.index,'latitude'] = imputed_locs.latitude
df.loc[imputed_locs.index,'longitude'] = imputed_locs.longitude

# mark the replaced ones
df['loc_estimated'] = False
df.loc[imputed_locs.index,'loc_estimated'] = True

df.loc[imputed_locs.index,['latitude','longitude']]

,latitude,longitude
sighting_idx,,
1,40.853524,-75.009542
32,40.853524,-75.009542
9,45.037929,-92.890117
11,43.427239,-70.670402
22,43.427239,-70.670402
...,...,...
4741,38.476217,-85.711122
4742,39.192585,-86.239410
4743,39.192585,-86.239410


In [37]:
df.loc[imputed_locs.index,['latitude','longitude']].isna().sum()

latitude     0
longitude    0
dtype: int64

## Clean dataframe then save

In [39]:
missingness(df)

observed               0.758374
location_details      15.546661
county                 0.000000
state                  0.000000
season                 0.000000
title                 20.012640
latitude               0.610912
longitude              0.610912
date                  20.012640
number                 0.000000
classification         0.000000
geohash               20.012640
temperature_high      34.737729
temperature_mid       37.855488
temperature_low       37.792290
dew_point             34.000421
humidity              34.000421
cloud_cover           39.898884
moon_phase            33.515905
precip_intensity      47.482621
precip_probability    47.482621
precip_type           67.137139
pressure              49.210027
summary               33.515905
uv_index              33.515905
visibility            40.657257
wind_bearing          33.705498
wind_speed            33.663366
loc_estimated          0.000000
dtype: float64

In [47]:
# drop remaining missing rows
df = df[~(df.latitude.isna() | df.longitude.isna())]

In [48]:
df.to_csv('sightings_with_estimates.csv', index=False)